In [1]:
import mechanize
import http.cookiejar
from bs4 import BeautifulSoup
import re

import pandas as pd
import time

import sys
sys.path.insert(0, '..')

from repository import *

In [2]:
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = http.cookiejar.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

# Script 1: Downloading Text

In [52]:
# Load old article urls to compare against
old_article_urls = pd.read_csv('article_urls.csv', names=['url'])
old_article_urls = set(old_article_urls['url'])

# Load previous page count
count = int(pd.read_csv('count.txt').columns[0])

# Load new urls
new_article_urls = load_new_urls(old_article_urls, count, n_pages=2)

Page 1: 0.8935160636901855
Page 2: 4.1816465854644775
4.182636022567749


In [71]:
start_time = time.time()

# Variables for new text, words, and phrases
new_text = ""
new_words = pd.Series(dtype='object')
new_two_word_phrases = pd.Series(dtype='object')
new_three_word_phrases = pd.Series(dtype='object')

# Mongolian alphabet
tolgoi = '[^АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоӨөПпРрСсТтУуҮүФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя ]'

for url, count in zip(new_article_urls, range(len(new_article_urls))):

    # Loaded html
    html = br.open(url).read()

    # Read html with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Find and print title to check we have loaded correctly
    print('Article ' + str(count) + ': ' + soup.find('title').string)
    
    # Extract text from the article
    text = soup.find('div', class_='has-content-area').get_text()
    new_text += " " + text
    
    # Add words to word list
    article_word_list = pd.Series(re.sub(tolgoi, ' ', text).lower().split())
    new_words = new_words.append(article_word_list, ignore_index=True)
    
    # Add phrases to phrase list
    two_word_phrases = (new_words.shift() + ' ' + new_words).dropna()
    three_word_phrases = (new_words.shift(2) + ' ' + new_words.shift() + ' ' + new_words).dropna()
    new_two_word_phrases.append(two_word_phrases, ignore_index=True)
    new_three_word_phrases.append(three_word_phrases, ignore_index=True)

time.time() - start_time

Article 0: Тэмээн аалзны төрлийн хүчтэй хортой, 20 насалдаг шинэ зүйлийг нээжээ | News.MN
Article 1: Эмнэлгээс гарч буй иргэд хувийн машинаар зорчиж болно | News.MN
Article 2: Орон сууц барихаар төлөвлөсөн газрын эзэмших эрхийг цуцаллаа | News.MN
Article 3: Хөлгийн багийн гишүүдийн "салах ёс гүйцэтгэсэн" бичлэг цацагдлаа | News.MN
Article 4: Жижиг, дунд бизнес эрхлэгчдийг цахимаар үнэ төлбөргүй сургана | News.MN
Article 5: Ерөнхийлөгч МАН-ыг тараах тухай холбогдох баримт, материалыг УДШ-д хүргүүллээ | News.MN
Article 6: ЭМЯ: 1098 тохиолдол батлагдаж, 51-78 насны найман хүн нас барлаа | News.MN
Article 7: "Биеийн байдал хөнгөн тохиолдолд өвөрмөц эмчилгээ хийхгүй" | News.MN
Article 8: Богдхан уулын урдуур тойрч Мааньт өртөөнд холбогдох төмөр замын төслийг хэлэлцэв | News.MN
Article 9: Эдийн засаг хямарсан ч олон улсын цэргийн төсөв рекорд тогтоов | News.MN
Article 10: С.Энхболд: Нас барсан 88 хүний гурав нь вакцин хийлгэсэн байсан | News.MN
Article 11: Цэргүүдийн хэргүүдийг хэрхэн шийдвэ

10.194899082183838

In [93]:
# Append text to text file
with open('text.txt', 'a') as f:
    f.write(new_text)

# Append word and phrase lists to respective csv's
new_words.to_csv('word_list.csv', mode='a', header=False, index=False)
new_two_word_phrases.to_csv('two_word_phrase_list.csv', mode='a', header=False, index=False)
new_three_word_phrases.to_csv('three_word_phrase_list.csv', mode='a', header=False, index=False)

# Record new urls in url file
pd.Series(list(new_article_urls)).to_csv('article_urls.csv', mode='a', header=False, index=False)

# Script 2: Word frequency, phrase frequency, example sentences

In [151]:
# Load text data
with open('text.txt', 'r') as f:
    text = f.read()
text = text.replace(u'\xa0', ' ')

# Load word list
word_list = pd.read_csv('word_list.csv', names=['Mongolian'])['Mongolian']

# Compute frequencies and save
frequency_list = pd.DataFrame({'Count': word_list.value_counts()})
frequency_list.index.rename('Word', inplace=True)
frequency_list.to_excel('frequency_list.xlsx')

# Find example sentences
for word in frequency_list.index:
    examples = re.findall(r"([^.]*?" + word + "[^.]*\.)", text)
    examples = list(filter (lambda example: (len (example) < 50) & (len (example) > 20), examples))[0:3]
    examples = "\n".join(examples)
    frequency_list.loc[word, 'Examples'] = examples

# Save examples
frequency_list.to_excel('frequency_list_with_examples.xlsx')

### Joining data from other dataframes

In [10]:
frequency_list = pd.read_excel('frequency_list.xlsx', engine='openpyxl')
df = frequency_list[:5000]

In [4]:
df = frequency_list[:15000]

### Bolor Toli

In [3]:
frequency_list_mtb = pd.read_excel('frequency_list_mtb_save.xlsx', engine='openpyxl')
wiktionary = pd.read_excel('../Wiktionary/wiktionary_common.xlsx', engine='openpyxl')

In [4]:
mtb_20000 = frequency_list_mtb[~frequency_list_mtb['Word'].isin(wiktionary['Mongolian'])]

In [5]:
mtb_20000

,Word,Count,ID,MTB Examples
11,буй,17823,16960,
12,л,15841,53960,NaN
14,аж,15343,2286,NaN
17,д,14135,28568,NaN
18,гэсэн,13205,28327,NaN
...,...,...,...,...
8499,шээг,3,135086,NaN
8500,хиртэх,3,108755,Муудаж хиртэх нь огт үгүй гэдгийг номын захын ...
8501,чигж,3,127501,NaN
8502,баяа,3,12957,NaN


In [10]:
mongolian = pd.Series(dtype='object')
translations = pd.Series(dtype='object')
examples_mn = pd.Series(dtype='object')
examples_en = pd.Series(dtype='object')
examples = pd.Series(dtype='object')

n = 100
for val in range(2600, len(mtb_20000), n):
    
    tr, ex_mn, ex_en, ex = bolor_toli_translate_word_list(br, mtb_20000.reset_index().loc[val:val+n-1, 'Word'])
    
    translations = translations.append(tr)
    examples_mn = examples_mn.append(ex_mn)
    examples_en = examples_en.append(ex_en)
    examples = examples.append(ex)

2600: чиглэлтэй
outright
2601: дэлбэрэх
ejection
2602: соронзон
magnetic
2603: бах
toad
2604: гэгээ
glow
2605: даг
disdain
2606: наана
before
2607: анализ
calculus
2608: ашиглуулах
to exploit
2609: хуш
cedar
2610: алан
intestines
2611: тийз
ticket
2612: ангиа
Not available
2613: үндэслэгч
founder
2614: ээдрээтэй
tangled
2615: эмгэнэлтэй
buskined
2616: хөлөгт
to wood
2617: таашаал
bang
2618: цуглуулга
collection
2619: хонох
to stay over
2620: хаших
to block
2621: тохиролцоо
deal
2622: баримжаа
orientation
2623: референт
adviser
2624: ингэх
do sth. in this manner
2625: оршуулах
committal
2626: ойл
Not available
2627: киоск
Not available
2628: гаралт
output
2629: сэр
rushing
2630: гэмтээх
disability
2631: цацах
to scatter
2632: фонд
resources
2633: ёо
Not available
2634: алжаал
tiredness
2635: тэнцвэрт
disbalance
2636: гаргалгаа
inference
2637: хөндөх
to involve
2638: төрхтэй
form
2639: мартагдашгүй
memorably
2640: концерт
concert
2641: хуурайшилт
extreme draught
2642: ирмэгт
maul
2643: б

somber
2944: бүхээг
shop
2945: устгал
Destruction
2946: зохиох
invent
2947: үзэсгэлэнтэй
beautiful
2948: малч
skilled in cattle-breeding
2949: замчин
scout
2950: жаргалан
jolly
2951: гамшигтай
calamitous
2952: асаргаа
nursing
2953: ноогдол
share
2954: бичилт
entry
2955: очирт
scepter
2956: номтой
scholarly
2957: барьцаагүй
Not available
2958: холболт
connecting
2959: ядам
annulary
2960: шингээх
suck
2961: илэн
openness
2962: хориод
around twenty
2963: агент
agent
2964: хөөр
frisk
2965: таамаглах
to assume
2966: хучилт
overlap
2967: ахуйд
services close to home
2968: түшиг
aid
2969: шагнах
premium
2970: төвөг
contretemps
2971: ханштай
responsive
2972: дөрөвхөн
only four
2973: гэнэтхэн
unexpectedly
2974: гүррагчаа
Not available
2975: бухимдах
depressing
2976: радикал
radical
2977: дуулиантай
covering a big territory
2978: уусмал
solution
2979: урсах
diffluence
2980: хэрэгцээгүй
to disqualify
2981: бригад
gang
2982: тэсрэлт
uproar
2983: ангижрах
to get rid of
2984: дугаарлах
to number
298

breed
3281: араншин
trait
3282: кабель
water-laid
3283: шинэчлэгдэх
to be renewed
3284: цоорхой
break
3285: булаах
to capture
3286: сэргээш
dope
3287: элсэлт
enrollment
3288: болц
maturity
3289: хай
slapdash
3290: сант
Not available
3291: ноогдох
hide land
3292: бүтэмжтэй
accomplished
3293: хийсвэр
ideal
3294: зохилтой
suitable
3295: эмт
Not available
3296: нурах
to fail
3297: дугуйтай
rolling
3298: ухамсарт
coconscious
3299: хүүхэн
wimp
3300: чингэлэгт
Not available
3301: тээх
to transport
3302: хоолтой
nourishing
3303: ижий
mama
3304: хэмжүүр
measure
3305: тууж
novelette
3306: хутаг
blessing
3307: түүчээ
the head wagon or animal in a caravan
3308: булхай
cheating dishonesty
3309: тэнцвэргүй
can't hold a candle to someone
3310: аажим
gradual
3311: өмсгөл
dress
3312: сахар
sugar
3313: хандивлагч
subscriber
3314: хайдаг
tinhorn
3315: арматур
reinforcing
3316: дээшгүй
Not available
3317: итгэлгүй
hopeless
3318: аваачих
to take away
3319: сэг
carrion
3320: өмчлөл
possession
3321: даатгалт

Not available
3621: дардан
spacious
3622: завших
take advantage of
3623: барилгатай
edificial
3624: битүүмжлэх
to seal
3625: экосистем
fragile ecosystem
3626: голчтой
trifocals
3627: бодон
boar
3628: давамгай
overweening
3629: сүрлэг
ambitious
3630: дүрэмгүй
having no procedure
3631: энхжин
to eden
3632: адам
Not available
3633: нүгэл
sin
3634: хонин
heath
3635: жаяг
law
3636: ахицтай
advanced
3637: даралттай
low-pressure
3638: исэл
oxide
3639: им
token
3640: нөгөөтээгүүр
on the other hand
3641: туршлагагүй
inexperienced
3642: намжих
to abate
3643: баривчлагдах
to cop out
3644: ургамалтай
grassy
3645: тушаа
solving
3646: гуулах
to put into a frame
3647: яаман
Not available
3648: хүртээмжгүй
Not available
3649: хатгасан
propulsive
3650: харанга
gong-bell
3651: горимтой
having procedures
3652: арчлах
to groom
3653: аялуулах
to organize a trip
3654: чангалах
to fasten
3655: харагдахуйц
perspicuous
3656: ханиадтай
tussive
3657: хуваагдах
fall into
3658: хангарди
Not available
3659: цаашгүй

to make easier
3963: ухаант
smart animal
3964: далдуур
on the Q.T.
3965: эндүүрэл
mal-entendus
3966: ажиллагаагүй
withered
3967: ганга
precipitous
3968: хүхэрлэг
sulfuric
3969: бэлэнчлэх
to prefer the ready-made things
3970: авхаалж
quickness
3971: хялбарчлах
to simplify
3972: намаг
marsh
3973: андуурах
be incorrect
3974: башир
simulation
3975: үүх
Not available
3976: ийнхэн
Not available
3977: хатгаа
pneumonia
3978: харалган
to jump
3979: арилжих
to do
3980: төвөө
Not available
3981: мөнгөжүүлэх
to turn into cash
3982: квант
quantum
3983: гагнуур
welding
3984: маргалдах
tug of love
3985: шоовдор
peon
3986: соло
solo
3987: түгшүүрт
to reduce tension
3988: жоу
Not available
3989: цөөрөх
or sparse
3990: итгэлт
allegiant
3991: өнтэй
warm
3992: доройтох
languish
3993: баясал
pleasure
3994: хэсэгхэн
partial
3995: замгүй
path
3996: сэрэмжлэх
anticipation
3997: шандун
Not available
3998: заль
artifice
3999: дэрвэгэр
dishevelled
4000: зовнил
anxiety
4001: хүчирхийлэх
outrace
4002: бусниулах
to

URLError: <urlopen error [Errno 104] Connection reset by peer>

In [11]:
translations

0             outright
1             ejection
2             magnetic
3                 toad
4                 glow
            ...       
95      to deteriorate
96               dwarf
97    to become active
98              single
99             applied
Length: 1500, dtype: object

In [12]:
test = pd.DataFrame({'English': translations, 'BT Examples MN': examples_mn, 'BT Examples EN': examples_en, 'BT Examples': examples})

In [9]:
test

,English,BT Examples MN,BT Examples EN,BT Examples
0,quantum,,,
1,embodied,,,
2,commander,ерөнхий командлагч,commander-in-chief,<p>ерөнхий командлагч: commander-in-chief</p>
3,mindset,,,
4,bird-cherry tree,,,
...,...,...,...,...
95,to be suspicious,,,
96,one-way,,,
97,abhorrent,зохимжгүй муу үгс,dirty words,<p>зохимжгүй муу үгс: dirty words</p>
98,sketch,зураглал хийх,to make a sketch,<p>зураглал хийх: to make a sketch</p>


In [14]:
mtb_20000.to_excel('test.xlsx', index=False)

In [31]:
mtb_20000 = mtb_20000.copy().drop(['English', 'BT Examples MN', 'BT Examples EN', 'BT Examples'], axis=1).reset_index(drop=True)

In [32]:
mtb_20000['English'] = test['English'].reset_index(drop=True)
mtb_20000['BT Examples MN'] = test['BT Examples MN'].reset_index(drop=True)
mtb_20000['BT Examples EN'] = test['BT Examples EN'].reset_index(drop=True)
mtb_20000['BT Examples'] = test['BT Examples'].reset_index(drop=True)

In [35]:
mtb_20000.drop('index', axis=1).to_excel('mtb_20000.xlsx', index=False)

In [13]:
frequency_list_mtb.to_excel('frequency_list_bt.xlsx', index=False)

# Join MTB data with frequency list

In [3]:
frequency_list = pd.read_excel('frequency_list.xlsx', engine='openpyxl')

In [4]:
mtb = pd.read_excel('frequency_list_mtb.xlsx', engine='openpyxl')

In [11]:
df = frequency_list.join(mtb.set_index('Word'), on='Word', rsuffix='_copy', how='right').drop('Count_copy', axis=1)

In [13]:
df.to_excel('frequency_list_mtb_save.xlsx', index=False)

# Join frequency list to Wiktionary

In [14]:
wiktionary = pd.read_excel('../Wiktionary/wiktionary_common.xlsx', engine='openpyxl')

In [19]:
df = wiktionary.join(frequency_list.set_index('Word'), on='Mongolian').sort_values(by='Count', ascending=False)

In [20]:
df.to_excel('../Wiktionary/wiktionary_common.xlsx', index=False)

# Sandbox